In [1]:
import os
import sys

In [3]:
sys.path.insert(0,os.path.join(os.getcwd(), '../'))
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from meteor import MeteorPatternScaling